# Lilly IPO Shard Runner (Colab)

This notebook runs a single shard of `build_ipo_pairs.py` and saves output to Google Drive.

**Before running:**
- Set `SHARD_ID` (00..07).
- Decide how you want to transfer the repo + artifacts to Colab (Git clone or Drive).

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Change this to your preferred Drive folder
OUT_DIR = '/content/drive/MyDrive/lilly_ipo'
!mkdir -p "{OUT_DIR}"

## Option A: Git clone repo
If your repo is accessible via git (public or with credentials), clone it below.
Otherwise use Option B to upload a zip or use Drive.

In [ ]:
# REPLACE with your repo URL
REPO_URL = 'YOUR_REPO_URL_HERE'
REPO_DIR = '/content/lilly'
# Uncomment if using git
# !git clone "{REPO_URL}" "{REPO_DIR}"

## Option B: Use Drive to provide artifacts
Place your repo or just the needed files in Drive and set `REPO_DIR` accordingly.
Required files:
- scripts/build_ipo_pairs.py
- experiments/affect_ipo/shards/prompts_shard_XX.json
- experiments/egist_32b/unified_lilly_probe.pt
- experiments/egist_32b/unified_dimensions.json
- experiments/lilly_subjective_32b/affect_classifier.pt
- experiments/lilly_subjective_32b/final_lilly_32b (adapter dir)

In [ ]:
# If using Drive, set REPO_DIR to the folder containing the repo/artifacts
REPO_DIR = '/content/drive/MyDrive/lilly_repo'  # change as needed

In [ ]:
!pip -q install transformers accelerate bitsandbytes peft

## (Optional) Hugging Face login
Only needed if you use gated/private models.

In [ ]:
# from huggingface_hub import login
# login()

In [ ]:
# Choose shard 00..07
SHARD_ID = '00'

# Run build_ipo_pairs for this shard
!python "{REPO_DIR}/scripts/build_ipo_pairs.py" \
  --input "{REPO_DIR}/experiments/affect_ipo/shards/prompts_shard_{SHARD_ID}.json" \
  --output "{OUT_DIR}/pairs_shard_{SHARD_ID}.jsonl" \
  --adapter-path "{REPO_DIR}/experiments/lilly_subjective_32b/final_lilly_32b" \
  --num-candidates 4 \
  --max-new-tokens 160 \
  --max-samples 100

## Done
Output will be saved to Google Drive at:
`/content/drive/MyDrive/lilly_ipo/pairs_shard_XX.jsonl`